In [ ]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/lgfquentin/dev/medical-dashboard/back-end-synthea-flink/config

In [ ]:
%load_ext streaming_jupyter_integrations.magics

In [ ]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081


In [ ]:
%%flink_execute_sql
CREATE TABLE encounter (
  `id` STRING,
  `status` STRING,
  `type` ROW (
    `coding` ROW (
      `code` STRING,
      `display` STRING
    ) ARRAY  ) ARRAY,
  `subject` ROW (
    `reference` STRING,
    `display` STRING
  ),
  `participant` ROW (
    `individual` ROW (
      `reference` STRING,
      `display` STRING
    )
  ) ARRAY,
  `period` ROW (
    `start` STRING,
    `end` STRING
  ),
  `serviceProvider` ROW (
    `reference` STRING
  ) 
) WITH (
  'connector' = 'kafka',
  'topic' = 'Encounter',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

In [ ]:
%%flink_execute_sql
CREATE TABLE visit_psql (
  `identifier` STRING,
  `id_organization` STRING,
  `id_practitioner` STRING,
  `id_patient` STRING,
  `status` STRING,
  `startTime` STRING,
  `endTime` STRING,
  `type` STRING,
  PRIMARY KEY (`identifier`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://db:5432/example',
  'table-name' = 'public.visit',
  'username' = 'example',
  'password' = 'example'
);

In [ ]:
%%flink_execute_sql
INSERT INTO visit_psql
SELECT 
id, 
status,
 e.type[1].coding[1].display as type,
 e.`period`.`start` as startTime,
  e.`period`.`end` as endTime,
   REPLACE(
    e.serviceProvider.reference, 
    'Organization?identifier=https://github.com/synthetichealth/synthea|', 
    ''
  ) AS id_organization,
  REPLACE(
      e.`subject`.`reference`, 
    'urn:uuid:', 
    ''
  )
  AS id_patient,
  REPLACE(
    e.participant[1].individual.reference, 
    'Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|', 
    ''
  ) AS id_practitioner
   FROM encounter e;